In [ ]:
import numpy as np
import time

from pct.hierarchy import PCTHierarchy
from pct.functions import Subtract
from pct.functions import Sigmoid
from pct.functions import Constant
from pct.functions import Proportional
from pct.functions import WeightedSum
from pct.functions import PassOn

from pct.functions import Integration
from pct.functions import OpenAIGym
from pct.functions import IndexedParameter
from pct.functions import GreaterThan

from pct.nodes import PCTNode
from pct.putils import FunctionsList

In [ ]:
cartpole_hierarchy = PCTHierarchy(levels=5, cols=1, name="cartpoleh", build=False)
cartpole_hierarchy.get_node(0, 0).name = 'cart_velocity'
cartpole_hierarchy.get_node(1, 0).name = 'cart_position'
cartpole_hierarchy.get_node(2, 0).name = 'pole_velocity'
cartpole_hierarchy.get_node(3, 0).name = 'pole_angle'
cartpole_hierarchy.get_node(4, 0).name = 'pole_position'

#cartpole = OpenAIGym("CartPole-v1", name="CartPole-v1", render=True)
cartpole = Constant([1,1,1,1,1], name="CartPole-v1")
cartpole_hierarchy.set_order("Down")


In [ ]:
cartpole_hierarchy.insert_function(level=0, col=0, collection="perception", function=IndexedParameter(index=1, name="cart_velocity", links=[cartpole]))
cartpole_hierarchy.insert_function(level=1, col=0, collection="perception", function=IndexedParameter(index=0, name="cart_position", links=[cartpole]))
cartpole_hierarchy.insert_function(level=2, col=0, collection="perception", function=IndexedParameter(index=3, name="pole_velocity", links=[cartpole]))
cartpole_hierarchy.insert_function(level=3, col=0, collection="perception", function=IndexedParameter(index=2, name="pole_angle", links=[cartpole]))
cartpole_hierarchy.insert_function(level=4, col=0, collection="perception", function=PassOn(name="pole_position", links=['cart_position']))


cartpole_hierarchy.insert_function(level=0, col=0, collection="reference", function=PassOn(name="cart_velocity_reference", links=['proportional1']))
cartpole_hierarchy.insert_function(level=1, col=0, collection="reference", function=PassOn(name="cart_position_reference", links=['proportional2']))
cartpole_hierarchy.insert_function(level=2, col=0, collection="reference", function=PassOn(name="pole_velocity_reference", links=['proportional3']))

cartpole_hierarchy.insert_function(level=0, col=0, collection="output", function=Integration(gain=-0.1, slow=4, name="force"))
#cartpole_hierarchy.summary()
cartpole_hierarchy.insert_function(level=4, col=0, collection="output", function=Sigmoid(range=0.45, scale=2, name="pole_position_output"))
#cartpole_hierarchy.summary(build=False)

cartpole_hierarchy.replace_function(level=3, col=0, collection="reference", function=PassOn(name="pole_angle_reference", links=['pole_position_output']), position=0)
cartpole_hierarchy.replace_function(level=2, col=0, collection="output", function=Integration(name="pole_velocity_output", gain=90, slow=100), position=0)
cartpole_hierarchy.set_links( 'cart_position_reference', 'pole_velocity_output')


In [ ]:
FunctionsList.getInstance().get_function("proportional3").set_name("pole_angle_output")
FunctionsList.getInstance().get_function("pole_angle_output").set_property('gain', 1.5)

#FunctionsList.getInstance().get_function("proportional2").set_name("pole_velocity_output")
#FunctionsList.getInstance().get_function("pole_velocity_output").set_property('gain', 0.5)

FunctionsList.getInstance().get_function("proportional1").set_name("cart_position_output")
FunctionsList.getInstance().get_function("cart_position_output").set_property('gain', 2)
FunctionsList.getInstance().get_function("constant4").set_name("pole_position_reference")
FunctionsList.getInstance().get_function("pole_position_reference").set_value(0)

In [ ]:
greaterthan = GreaterThan(threshold=0, upper=1, lower=0, links='force')
cartpole_hierarchy.add_postprocessor(greaterthan)
cartpole_hierarchy.add_preprocessor(cartpole)
cartpole.add_link(greaterthan)

cartpole_hierarchy.summary(build=True)
#FunctionsList.getInstance().report()


cartpoleh PCTHierarchy
**************************
PRE: CartPole-v1 Constant | [1, 1, 1, 1, 1] | links  greaterthan 
Level 4
pole_position PCTNode
----------------------------
REF: pole_position_reference Constant | 0 
PER: pole_position PassOn | 0 | links  cart_position 
COM: subtract4 Subtract | 0 | links  pole_position_reference pole_position 
OUT: pole_position_output Sigmoid | range 0.45 scale 2  | 0 | links  subtract4 
----------------------------
Level 3
pole_angle PCTNode
----------------------------
REF: pole_angle_reference PassOn | 0 | links  pole_position_output 
PER: pole_angle IndexedParameter | index 2 | 0 | links  CartPole-v1 
COM: subtract3 Subtract | 0 | links  pole_angle_reference pole_angle 
OUT: pole_angle_output Proportional | gain 1.5 | 0 | links  subtract3 
----------------------------
Level 2
pole_velocity PCTNode
----------------------------
REF: pole_velocity_reference PassOn | 0 | links  pole_angle_output 
PER: pole_velocity IndexedParameter | index 3 | 0 | l

In [ ]:
#cartpole_hierarchy.draw(font_size=10, figsize=(8,12), move={'CartPole-v1': [-0.075, 0] , 'pole_velocity': [-0.02, 0], 'pole_angle': [-0.025, 0]}, node_size=1000)


In [ ]:
cartpole_hierarchy.run(10, verbose=True)
#cartpole_hierarchy.run(100)

['1.000', '1.000', '1.000', '1.000', '1.000'] 
pole_position 0.000 0.000 0.000 0.000 
pole_angle 0.000 1.000 -1.000 -1.500 
pole_velocity -1.500 1.000 -2.500 -2.250 
cart_position -2.250 1.000 -3.250 -6.500 
cart_velocity -6.500 1.000 -7.500 0.188 
1.000 

['1.000', '1.000', '1.000', '1.000', '1.000'] 
pole_position 0.000 1.000 -1.000 -0.220 
pole_angle -0.220 1.000 -1.220 -1.830 
pole_velocity -1.830 1.000 -2.830 -4.774 
cart_position -4.774 1.000 -5.774 -11.548 
cart_velocity -11.548 1.000 -12.548 0.454 
1.000 

['1.000', '1.000', '1.000', '1.000', '1.000'] 
pole_position 0.000 1.000 -1.000 -0.220 
pole_angle -0.220 1.000 -1.220 -1.830 
pole_velocity -1.830 1.000 -2.830 -7.273 
cart_position -7.273 1.000 -8.273 -16.546 
cart_velocity -16.546 1.000 -17.546 0.779 
1.000 

['1.000', '1.000', '1.000', '1.000', '1.000'] 
pole_position 0.000 1.000 -1.000 -0.220 
pole_angle -0.220 1.000 -1.220 -1.830 
pole_velocity -1.830 1.000 -2.830 -9.747 
cart_position -9.747 1.000 -10.747 -21.494 
cart

1

In [ ]:
cartpole.close()